In [1]:
import pandas as pd
import timestring 
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from sklearn import linear_model
import warnings
warnings.filterwarnings("ignore")

## Load the data and remove all rows that are missing a value for ANY of the following columns: 'Title', 'Artist', or 'DateAcquired'. Use this cleaned data for the rest of the challenge. After removing these rows, how many rows remain?

In [2]:
artworks= pd.read_csv("C:/Users/rahaf/Downloads/artworks.csv")
print('The artwork dataset originally had', format(artworks.shape[0],','),'rows.')
print()
artworks_clnd=artworks.dropna(how='any', subset=['Title', 'Artist','DateAcquired'])
print('The artwork dataset after cleaning has', format(artworks_clnd.shape[0],','),'rows.')

The artwork dataset originally had 138,151 rows.

The artwork dataset after cleaning has 129,930 rows.


## Looking at the Title for each work of art, how many unique titles contain the string 'untitled' (NOT case-sensitive)?

In [3]:
#list the unique values of the column Title
title_list=artworks_clnd['Title'].unique().tolist() 

#lowercasing the title_list
title_list_l=list(map(str.lower, title_list))

#find titles in title_list_l that include the string 'untitled' 
title_with_substring= [title for title in title_list_l if 'untitled' in title]
print('number of unique titles that contain "untitled" (NOT case-sensitive) =',len(title_with_substring))

number of unique titles that contain "untitled" (NOT case-sensitive) = 3900


## Of the works with a non-null, positive value for Duration (column 'Duration (sec.)'), what proportion of these durations are listed as being longer than 10 hours?

In [4]:
a=artworks_clnd[artworks_clnd['Duration (sec.)']>0]#positive values
b=a[a['Duration (sec.)']>36000]
print('a=',a.shape[0], 'b=', b.shape[0], 'b/a=', b.shape[0]/a.shape[0])


a= 1844 b= 24 b/a= 0.013015184381778741


## The Golden Ratio (phi ~= 1.618) is popularly regarded for its aesthetic qualities, but how typically is it observed in canvas shapes in MoMA's collection? For all works in the department of Drawings & Prints whose Width (column 'Width (cm)') and Height (column 'Height (cm)') are greater than zero, compute each piece's aspect ratio as Width / Height. What proportion of these aspect ratios are within one percent of the Golden Ratio or the inverse of the Golden Ratio (which would denote a vertical Golden Rectangle)?

In [5]:
#filter the data
filtered_data=artworks_clnd[(artworks_clnd['Department']=='Drawings & Prints') &
              (artworks_clnd['Width (cm)']>0) &
              (artworks_clnd['Height (cm)']>0)].copy()

#compute width height ratio
filtered_data['w/h']=filtered_data['Width (cm)']/ filtered_data['Height (cm)']

#1 if ratio is within the accepted range (inclusive), 0 ow
filtered_data['GR']=filtered_data['w/h'].apply(lambda x: 1 if 
                                               (1.608<=x<=1.628)|(0.608<=x<=0.628)
                                               else 0)
#compute the proportion of the w/h ratios 
filtered_data[filtered_data['GR']==1].shape[0]/filtered_data.shape[0]

0.016048764686828233

## Of all pieces listed, what proportion belong to the Photography Department? Remember, you should be using the cleaned data you prepared in the last question.

In [6]:
artworks_clnd[artworks_clnd['Department']=='Photography'].shape[0]/artworks_clnd.shape[0]

0.23909027938120525

## MoMA tends more heavily toward printed works, rather than painted works, as evidenced by the value counts for each Department in the data set. What about the materials on which these prints and paintings appear? Looking at the Medium for each work of art, report the ratio of the number of works that contain the string 'paper' compared to those that contain the string 'canvas' (NOT case-sensitive).

In [7]:
artworks_clnd['Department'].unique()

array(['Architecture & Design', 'Drawings & Prints',
       'Painting & Sculpture', 'Photography', 'Media and Performance',
       'Film', 'Fluxus Collection'], dtype=object)

In [8]:
print_paint=artworks_clnd[(artworks_clnd['Department']=='Drawings & Prints') |
              (artworks_clnd['Department']=='Painting & Sculpture')]

# find works that include the string 'paper' 
print_paint_paper= print_paint['Medium'].str.lower().str.contains(r'paper')

# find works that include the string 'canvas'
print_paint_canvas= print_paint['Medium'].str.lower().str.contains(r'canvas')

print(' ratio of the number of works that contain the string "paper" compared to those that contain the string "canvas" (NOT case-sensitive)', 
      print_paint_paper.sum()/print_paint_canvas.sum())

 ratio of the number of works that contain the string "paper" compared to those that contain the string "canvas" (NOT case-sensitive) 7.02985884907709


## For each year between 1960-1970 (inclusive), compute the number of unique artists whose work MoMA acquired that year. Perform a linear regression on these counts, where X is the year and y is the count of unique artists. Report the R^2 value for this fitted regression model.

In [9]:
artworks_clnd.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

In [10]:
#convert datetime string to year
artworks_clnd['YearAcquired']=artworks_clnd['DateAcquired'].apply(lambda x:timestring.Date(x).year)

In [11]:
#filter dates
date_filtered=artworks_clnd[(artworks_clnd['YearAcquired']<=1970) &(artworks_clnd['YearAcquired']>=1960)]

#group unique artists by year
date_prediction=pd.DataFrame(date_filtered.groupby(['YearAcquired'])['Artist'].unique())
date_prediction.reset_index(level=0, inplace=True)

#compute count of unique artists in each year
date_prediction['CountUniqueArtists']=date_prediction['Artist'].apply(lambda x: len(x)) 

date_prediction

,YearAcquired,Artist,CountUniqueArtists
0,1960,"[Richard Riemerschmid, Hector Guimard, Louis C...",224
1,1961,"[August Endell, Peter Behrens, Joseph Maria Ol...",252
2,1962,"[Tapio Wirkkala, Sheila Hicks, Percy Stamp, Na...",167
3,1963,"[Hans Hollein, Walter Pichler, Ludwig Mies van...",309
4,1964,"[Louis I. Kahn, Ludwig Mies van der Rohe, Tapi...",653
5,1965,"[Raimund Abraham, Kenneth Snelson, Ludwig Mies...",417
6,1966,"[Hans Poelzig, Paul Nelson, Frantz Jourdain, O...",326
7,1967,"[Louis I. Kahn, Ann Tyng, Louis I. Kahn, Hans ...",578
8,1968,"[Ludwig Mies van der Rohe, Raimund Abraham, Ca...",451
9,1969,"[Jason Crum, Konrad Wachsmann, Mary Walker Phi...",445


In [12]:
#linear regression
lm = linear_model.LinearRegression()
x=pd.DataFrame(date_prediction['YearAcquired'])
y=pd.DataFrame(date_prediction['CountUniqueArtists'])
reg = lm.fit(x,y)
print('The coefficient of determination R^2 of the prediction is',reg.score(x,y))

The coefficient of determination R^2 of the prediction is 0.1706237304148105


## Does the number of MoMA's yearly acquisitions vary along with changes to the market? Compute the number of pieces acquired by MoMA for each year from 2000-2010 (inclusive), then compute the percent difference in yearly acquisitions. Next, using values from the link provided, obtain January's average closing price for the S&P 500 across those same years (2000-2010). Compute the year-to-year percent difference of those values, as well. Finally, compute and report the Pearson's correlation coefficient between the percent differences in yearly acquisitions and in the average January S&P 500 closing prices.

In [13]:
#filter the year
date_filtered2=artworks_clnd[(artworks_clnd['YearAcquired']<=2010) &
                             (artworks_clnd['YearAcquired']>=2000)]

#find nuber of pieces acquired each year
yearly_aqcuisitions=pd.DataFrame(date_filtered2.groupby(['YearAcquired'])
                                 ['ConstituentID'].count())

yearly_aqcuisitions.reset_index(level=0, inplace=True)

yearly_aqcuisitions.rename(columns={'ConstituentID': 'TotalPieces'},
                           inplace='True')

In [14]:
yearly_aqcuisitions['pct_change']=yearly_aqcuisitions['TotalPieces'].pct_change()

In [15]:
yearly_aqcuisitions

,YearAcquired,TotalPieces,pct_change
0,2000,1880,NaN
1,2001,4160,1.212766
2,2002,651,-0.843510
3,2003,549,-0.156682
4,2004,763,0.389800
5,2005,4030,4.281782
6,2006,1666,-0.586600
7,2007,2342,0.405762
8,2008,6518,1.783091
9,2009,1536,-0.764345


In [16]:
# use BeautifulSoup to scrape the S&P price and year from the link
url = "https://www.multpl.com/s-p-500-historical-prices/table/by-year"
html = urlopen(url)

# create a Beautiful Soup object from the html
soup = BeautifulSoup(html, 'lxml')
type(soup)

# the class of the table part of the page:
website_text = soup.find('div', {'class' :'tcol'}).text

#convert string to list
My_list= re.findall('(.+?)\n' , website_text)
My_list

['Date',
 'Price',
 'Value',
 'Jan 14, 2021',
 '3,812.66',
 'Jan 1, 2021',
 '3,744.86',
 'Jan 1, 2020',
 '3,278.20',
 'Jan 1, 2019',
 '2,607.39',
 'Jan 1, 2018',
 '2,789.80',
 'Jan 1, 2017',
 '2,275.12',
 'Jan 1, 2016',
 '1,918.60',
 'Jan 1, 2015',
 '2,028.18',
 'Jan 1, 2014',
 '1,822.36',
 'Jan 1, 2013',
 '1,480.40',
 'Jan 1, 2012',
 '1,300.58',
 'Jan 1, 2011',
 '1,282.62',
 'Jan 1, 2010',
 '1,123.58',
 'Jan 1, 2009',
 '865.58',
 'Jan 1, 2008',
 '1,378.76',
 'Jan 1, 2007',
 '1,424.16',
 'Jan 1, 2006',
 '1,278.73',
 'Jan 1, 2005',
 '1,181.41',
 'Jan 1, 2004',
 '1,132.52',
 'Jan 1, 2003',
 '895.84',
 'Jan 1, 2002',
 '1,140.21',
 'Jan 1, 2001',
 '1,335.63',
 'Jan 1, 2000',
 '1,425.59',
 'Jan 1, 1999',
 '1,248.77',
 'Jan 1, 1998',
 '963.36',
 'Jan 1, 1997',
 '766.22',
 'Jan 1, 1996',
 '614.42',
 'Jan 1, 1995',
 '465.25',
 'Jan 1, 1994',
 '472.99',
 'Jan 1, 1993',
 '435.23',
 'Jan 1, 1992',
 '416.08',
 'Jan 1, 1991',
 '325.49',
 'Jan 1, 1990',
 '339.97',
 'Jan 1, 1989',
 '285.40',
 'Jan 1,

In [17]:
#remove the first 5 unuseful elements of the list
del My_list[0:5]

#create a list of all years
year_list= My_list[0::2]

#remove the string next to the year numbers
year_list=[s.replace('Jan 1, ', '') for s in year_list]

#create a list of all prices
price_list= My_list[1::2]
price_list=[s.replace(',', '') for s in price_list]

year_list=map(int,year_list)
price_list=map(float,price_list)

#create the year-price table
sp_table = pd.DataFrame({'Year':year_list,'Price':price_list})

In [18]:
#filter the year
sp_table=sp_table[(sp_table['Year']<=2010) &(sp_table['Year']>=2000)].copy()

sp_table['sp_pct_change']=sp_table['Price'].pct_change()
sp_table.reset_index(drop=True , inplace=True)
sp_table

,Year,Price,sp_pct_change
0,2010,1123.58,NaN
1,2009,865.58,-0.229623
2,2008,1378.76,0.592874
3,2007,1424.16,0.032928
4,2006,1278.73,-0.102116
5,2005,1181.41,-0.076107
6,2004,1132.52,-0.041383
7,2003,895.84,-0.208985
8,2002,1140.21,0.272783
9,2001,1335.63,0.171389


In [19]:
print('the Pearson correlation coefficient between the percent differences in yearly acquisitions and in the average January S&P 500 closing prices is',
      yearly_aqcuisitions['pct_change'].corr(sp_table['sp_pct_change'] , method='pearson'))

the Pearson correlation coefficient between the percent differences in yearly acquisitions and in the average January S&P 500 closing prices is -0.32643908947050837
